In [1]:
# !pip install gensim --user

In [1]:
from gensim.models import Word2Vec

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [3]:
import re
import numpy as np
import pandas as pd
from string import punctuation
from PreProcessing.Opentext import get_EBS

['lec00001_EBS.txt', 'lec00002_EBS.txt', 'lec00003_EBS.txt', 'lec00004_EBS.txt', 'lec00005_EBS.txt', 'lec00006_EBS.txt', 'lec00007_EBS.txt', 'lec00008_EBS.txt', 'lec00009_EBS.txt', 'lec00010_EBS.txt', 'lec00011_EBS.txt', 'lec00012_EBS.txt', 'lec00013_EBS.txt', 'lec00014_EBS.txt', 'lec00015_EBS.txt', 'lec00016_EBS.txt', 'lec00017_EBS.txt', 'lec00018_EBS.txt', 'lec00019_EBS.txt', 'lec00020_EBS.txt', 'lec00021_EBS.txt', 'lec00022_EBS.txt', 'lec00023_EBS.txt', 'lec00024_EBS.txt', 'lec00025_EBS.txt', 'lec00026_EBS.txt']


In [4]:
from jamo import h2j, j2hcj, j2h
from soynlp.hangle import levenshtein, jamo_levenshtein
import nltk
from nltk import Text
from nltk.tag import pos_tag
from nltk.corpus import gutenberg
from nltk.tokenize import sent_tokenize, word_tokenize, TweetTokenizer, regexp_tokenize
from nltk.help import upenn_tagset
from konlpy.corpus import kolaw, kobill
from konlpy.tag import Kkma, Mecab, Okt, Komoran, Hannanum
from PreProcessing.find_common_part import find_common_part
from PreProcessing.Dictionary import sd, ad, kopro
from PreProcessing.Opentext import get_EBS, get_EBS_entered, get_STT1_Google_entered, get_STT1_Google, get_STT1_Transcribe_entered
from collections import defaultdict
pd.set_option("display.max_rows", None, "display.max_columns", None)
from konlpy.tag import Kkma, Mecab, Okt, Komoran, Hannanum

0
0
0
1
2
5
4
8


In [5]:
EBS = get_EBS(1, 26)

['lec00001_EBS.txt', 'lec00002_EBS.txt', 'lec00003_EBS.txt', 'lec00004_EBS.txt', 'lec00005_EBS.txt', 'lec00006_EBS.txt', 'lec00007_EBS.txt', 'lec00008_EBS.txt', 'lec00009_EBS.txt', 'lec00010_EBS.txt', 'lec00011_EBS.txt', 'lec00012_EBS.txt', 'lec00013_EBS.txt', 'lec00014_EBS.txt', 'lec00015_EBS.txt', 'lec00016_EBS.txt', 'lec00017_EBS.txt', 'lec00018_EBS.txt', 'lec00019_EBS.txt', 'lec00020_EBS.txt', 'lec00021_EBS.txt', 'lec00022_EBS.txt', 'lec00023_EBS.txt', 'lec00024_EBS.txt', 'lec00025_EBS.txt', 'lec00026_EBS.txt']


In [6]:
GOOGLE = get_STT1_Google(1, 26)

['lec00001_STT1_Google.txt', 'lec00002_STT1_Google.txt', 'lec00003_STT1_Google.txt', 'lec00004_STT1_Google.txt', 'lec00005_STT1_Google.txt', 'lec00006_STT1_Google.txt', 'lec00007_STT1_Google.txt', 'lec00008_STT1_Google.txt', 'lec00009_STT1_Google.txt', 'lec00010_STT1_Google.txt', 'lec00011_STT1_Google.txt', 'lec00012_STT1_Google.txt', 'lec00013_STT1_Google.txt', 'lec00014_STT1_Google.txt', 'lec00015_STT1_Google.txt', 'lec00016_STT1_Google.txt', 'lec00017_STT1_Google.txt', 'lec00018_STT1_Google.txt', 'lec00019_STT1_Google.txt', 'lec00020_STT1_Google.txt', 'lec00021_STT1_Google.txt', 'lec00022_STT1_Google.txt', 'lec00023_STT1_Google.txt', 'lec00024_STT1_Google.txt', 'lec00025_STT1_Google.txt', 'lec00026_STT1_Google.txt']


In [7]:
EBS[1070:3000]

'게 수선을 똑같이 그어 주면 바로 두 직각삼각형 ABC하고, ADE는 서로 어떤 닮음?AA닮음이 된다. 이건 우리가 중학교 2학년 때 배웠죠, 그렇죠? ∠A가 공통이고, 그다음에 둘 다 직각을 갖고 있으니까 AA닮음이 될 테고.우리가 중2 때 배웠던 닮음비라는 건 뭐였냐 하면 대응하는 변의 길이의 비였어요, 그렇죠? 그래서 만약에 이 길이를 5라고 해 보고, 이 길이를 10이라고 해 보면.두 삼각형의 닮음비는 5:10. 1:2겠죠. 그렇다면 우리가 이 길이를 4라고 놔 보면, 이 길이는 얼마가 됩니까, 당연히? 그렇죠. 4의 두 배인 8이 돼야 돼.이 길이를 거꾸로 우리가 6이라고 해 보겠습니다. 그럼 거꾸로 이 작은 쪽은 얼마가 될까? 1:2, 2:1. 얼마? 3이 돼야 된다는 걸 여러분들이 알 수 있죠.그래서 우리는 닮음비라는 건 ‘작은 삼각형과 큰 삼각형의 대응하는 변의 길이의 비’이니까 5:10, 3:6, 4:8. 이것이 모두 다 1:2로 같고 그것을 닮음비라고 한다.이렇게 공부를 했었는데 이것 말고도 이러한 것들도 성립할 거란 얘기예요. 예를 들어 작은 삼각형에서 빗변 분의 높이 얼마입니까? 5분의3이죠?큰 삼각형에서 빗변 분의 높이 얼마죠? 10분의6이죠. 이 둘 어때요? 5분의3으로 서로 같다는 거 확인할 수 있고.또 마찬가지로 작은 삼각형에서 예를 들어서 빗변 분의 밑변 해 봅시다. 5분의4죠. 큰 삼각형에서 빗변 분의 밑변 해 보면 얼마죠?10분의8이죠. 근데 이것도 서로 어때? 같아요. 마찬가지로 하나 더 해 볼까요? 작은 삼각형에서 밑변 분의 높이는 4분의3이죠.큰 삼각형에서 밑변 분의 높이는 어때요? 8분의6인데, 이들이 모두 다 어때? 같다는 걸 여러분들이 알 수 있단 얘기예요. 결국 지금 이 둘의 차이는, 같은 거 같아 보이나요? 조금 달라요.작은 삼각형과 큰 삼각형의 두 대응변의 길이의 비를 이야기하는 게 닮음비고, 한 삼각형에서 어떤 두 변을 뽑아서 그 둘의 비랑 큰 삼각형에서 그 둘의 길이의 비는비가 서로 같은 성질이 있다는 게

In [8]:
GOOGLE[1000:3000]

' 수선을 똑같이 보여 주면 바로 2차각성 가평 a b a d는 서로 어떤 닮은 a 닮음이 된다 이건 우리가 중학교 2학년 때 가게가 통이고 그다음에 둘 다 치과를 갖고 있으니까 a 삶이 될 테고 우리가 주의 때문에 어떤 닮음비 일하는 건 뭐냐면 대응하는 변의 길이의 비 왔어요 그래서 그래서 만약에 이게들을 오라고 해 보고 얘기를 11분 삼각형의 닮음 비는 5대 11대 이겠죠 자 그렇다면 우리가이 길을 사라고 나 보면 이게 된 얼마나 됩니까 당연히 그쵸 5일 8일 해야 돼 진짜이 길을 걷고 우리가 6이라고 해보겠습니다 그럼 거꾸로 작은 쪽은 얼마나 됐다 1대 2대 1 얼마 3일 돼야 된다는 걸 여러분들이 할 수 있죠\n 우리는 닮은 비라는 건 작은 삼각형과 큰 삼각형의 대응하는 변의 길이의 비니까 오대식 3대 6\n 4대 8 이것이 모두 다 일대일로 갖고 그것을 닮음비라고 한다 이렇게 공부했었는데 이것 말고도 이런 것들도 성립할 거라 예를 들어 작은 삼각형에서 빗변분의 높이 얼마입니까 큰 삼각형에서 빗변분의 높이 얼마죠 숯불에 유기적 이들 어때요 5분의 3으로 서로 같다라는 확인할 수 있고 또 마찬가지로 자는 삼각형에서 예를 들어서 빗변분의 밑변에 봅시다 5분의 4 적 큰 삼각형에서 변 분에 밑변에 보면 얼마죠 10분에 반이죠 근데 이것도 사러 때 같아요 마찬가지로 하나 더 해 볼까요 작은 삼각형에서 밑변 불에 높이는 4분의 3 이적 큰 삼각형에서 미평 그래도 필요 없대요 8분의 6인데 이들이 모두 다 어때 같다란 걸려 놈들이 알 수 있단 얘기야 결국 지금이 둘의 차이는 같은 거 같아 보이나요\n 달라요 작은 삼각형과 큰 삼각형의 두 대는 변의길이 비를 이야기 한게 닮음비고 1 삼각형에서 어떤 두 병을 뽑아서 그들의 비람 큰 삼각형 둘레 길이 8m 서로 비가 서로 같은 성질이 있다라는게 바로 닮은 직각삼각형에서 성질입니다 이제 우리 가요 내용은 당연히 받아들일 수 있을 테니이 내용에 대해서 간단하게 확인 문자 한번 살펴보고 가도록 하겠습니다 오른쪽 그림과 같은

In [9]:
def decimalnumber_to_korean(m):
    answer = ''
    for _ in m.group(1):
        answer += kopro[int(_)]
    answer += '점'
    for _ in m.group(3):
        answer += kopro[int(_)]
    return answer

In [10]:
def number_to_korean(s):
    #소수점 한국어로
    s = re.sub(r'(\d+)(\.)(\d+)', decimalnumber_to_korean, s)
    
    #정수 한국어로
    fl = re.findall(r'\d+', s)
    fl = set(fl)
    fl = [int(_) for _ in fl]
    fl.sort(reverse=True)
    fl = [str(_) for _ in fl]
    for n in fl:
        s = s.replace(n, kopro[int(n)])
    return s

In [11]:
def mathsymbol_to_korean(s):
    for ms in sd.keys():
        if ms in s:
            s = s.replace(ms, sd[ms])
    return s

In [12]:
def alphabet_to_korean(s):
    for a in ad.keys():
        if a in s:
            s = s.replace(a, ad[a])
    return s

In [13]:
def remove_newlines(s):
    
    return re.sub(r'[\n]+', ' ', s)

In [14]:
def remove_puncuations(s):
    
    return re.sub(r'[\.?,‘’]+', ' ', s)

In [15]:
def insert_dot(text, dot):
    pattern1 = re.compile(r'(ㅂㄴㅣㄷㅏ)$')   #ㅂ 니다
    pattern2 = re.compile(r'(.ㅔ|ㅏ|ㅓ|ㅐ|ㅗ|ㅜ)(ㅇㅛ)$') # ㅔ요, ㅏ요, ㅓ요
#     pattern3 = re.compile(r'(ㅆ)(ㅈ|ㅊ)(ㅛ)$') 
    pattern4 = re.compile(r'(ㅂ|ㅣ)(ㄴㅣㄲㅏ)$')  #ㅂ니까
    pattern5 = re.compile(r'(ㄴㄷㅔ)$')
    text_list = []
    _1 = 0
    _2 = 0
    _3 = 0
    _4 = 0
    _5 = 0
    _6 = 0
    for _ in text.split(' '):         
        new_ = j2hcj(h2j(_))
        if pattern1.findall(new_):
            text_list.append(_.replace(_, _+dot))
            _1 += 1
        elif pattern2.findall(new_):
            text_list.append(_.replace(_, _+dot))
            _2 += 1
            #list2.append(_)
#         elif pattern3.findall(new_):
#             text_list.append(_.replace(_, _+dot))
#             _3 += 1
#             list3.append(_)
        elif pattern4.findall(new_):
            text_list.append(_.replace(_, _+dot))
            _4 += 1
        elif pattern5.findall(new_):
            text_list.append(_.replace(_,_+dot))
            _5 += 1
        else:
            text_list.append(_)
            _6 += 1
            #list6.append(_)
    print('pattern1 = {}, pattern2= {}, pattern3 = {}, pattern5 = {}, pattern6 = {}'.format(_1,_2,_3,_4,_5,_6)) 
    #print('pattern2 = ',list2)#'\n','list5 = ',list5)
    return text_list    

In [18]:
# EBS 전처리
def EBS_preprocessing(EBS):
    
#     수학기호 한국어로 변환(EBS)
    ebs_math_conversion = mathsymbol_to_korean(EBS)
        
#     숫자 한국어로 변환(EBS)
    ebs_number_conversion = number_to_korean(ebs_math_conversion)
        
#     알파벳 한국어로 변환(EBS)
    ebs_alphabet_conversion = alphabet_to_korean(ebs_number_conversion)
    
#     특수문자 제거
    ebs_punc_removed = remove_puncuations(ebs_alphabet_conversion)
    
#     점 집어넣기
    ebs_dot_inserted = ' '.join(insert_dot(ebs_punc_removed,'.'))

#     띄어쓰기 제거
    ebs_whitespace_removed = re.sub('[\s]+', '', ebs_dot_inserted)

    return ebs_whitespace_removed

In [19]:
# GOOGLE 전처리
def GOOGLE_preprocessing(GOOGLE):
        
#     개행 삭제
    google_newline_removed = remove_newlines(GOOGLE)
    
#     수학기호 한국어로 변환
    google_math_conversion = mathsymbol_to_korean(google_newline_removed)
    
#     숫자 한국어로 변환(GOOGLE)
    google_number_conversion = number_to_korean(google_math_conversion)
        
#     알파벳 한국어로 변환(GOOGLE)
    google_alphabet_conversion = alphabet_to_korean(google_number_conversion)
    
#     특수문자 제거
    google_punc_removed = remove_puncuations(google_alphabet_conversion)
    
#     점 집어넣기
    google_dot_inserted = ' '.join(insert_dot(google_punc_removed,'.'))
    
#     띄어쓰기 제거
    google_whitespace_removed = re.sub('[\s]+', '', google_dot_inserted)

    return google_whitespace_removed

In [20]:
ko_pre_EBS = EBS_preprocessing(EBS) #한글로 치환

pattern1 = 1872, pattern2= 2455, pattern3 = 0, pattern5 = 781, pattern6 = 1128


In [21]:
ko_pre_EBS[1000:2000]

'이돼야돼이길이를거꾸로우리가육이라고해보겠습니다.그럼거꾸로이작은쪽은얼마가될까일대이이대일얼마삼이돼야된다는걸여러분들이알수있죠그래서우리는닮음비라는건작은삼각형과큰삼각형의대응하는변의길이의비이니까.오대십삼대육사대팔이것이모두다일대이로같고그것을닮음비라고한다이렇게공부를했었는데.이것말고도이러한것들도성립할거란얘기예요예를들어작은삼각형에서빗변분의높이얼마입니까.오분의삼이죠큰삼각형에서빗변분의높이얼마죠십분의육이죠이둘어때요.오분의삼으로서로같다는거확인할수있고또마찬가지로작은삼각형에서예를들어서빗변분의밑변해봅시다오분의사죠큰삼각형에서빗변분의밑변해보면얼마죠십분의팔이죠근데.이것도서로어때같아요.마찬가지로하나더해볼까요.작은삼각형에서밑변분의높이는사분의삼이죠큰삼각형에서밑변분의높이는어때요.팔분의육인데.이들이모두다어때같다는걸여러분들이알수있단얘기예요결국지금이둘의차이는같은거같아보이나요.조금달라요.작은삼각형과큰삼각형의두대응변의길이의비를이야기하는게닮음비고한삼각형에서어떤두변을뽑아서그둘의비랑큰삼각형에서그둘의길이의비는비가서로같은성질이있다는게바로닮은직각삼각형에서갖는성질입니다.그렇다면이제우리가이내용은당연히받아들일수있을테니이내용에대해서간단하게확인문제한번살펴보고가도록하겠습니다.오른쪽그림과같은삼각형에이비씨안에있는작은삼각형큰삼각형에이디이가있습니다.다음을구하세요.라고했는데우리큰삼각형에선지금길이를알수가없어요.그렇죠작은삼각형만주어져있는데.구하라는건뭔가봤더니에이이분의디이그려볼까요.에이이분의디이가뭘까에이이분의디이큰삼각형에서의빗변분의높이를구하라는얘기네근데.그건우리가결국누구를구하는것과같다작은삼각형에서의빗변분의높이와서로어때같다는이야기입니다.왜두직각삼각형은서로닮음이니까.그럼결국이건뭐라고쓸수있어요.바로가볼게요.빗변분의높이였으니빗변분의높이이렇게해주면루트이분의일이될테고유리화해서이분의루트이이렇게여러분들이구할수있다는얘기입니다.하나더두번째거뭘구하라고했습니까.에이이분의에이디이죠에이이분의에이디빗변분의밑변이네나와있진않지만우린구할수있다작은직각삼각형에서빗변분의밑변을해주면되니까루트이분의일이구나이렇게여러분들께서구해서유리화까지만해주면바로답을구할수있다는이야기예요어렵나요.아직까지내

In [22]:
ko_pre_GOOGLE = GOOGLE_preprocessing(GOOGLE) #한글로 치환

pattern1 = 1729, pattern2= 2167, pattern3 = 0, pattern5 = 563, pattern6 = 1109


In [23]:
ko_pre_GOOGLE[1000:2000]

'얼마삼일돼야된다는걸여러분들이할수있죠우리는닮은비라는건작은삼각형과큰삼각형의대응하는변의길이의비니까.오대식삼대육사대팔이것이모두다일대일로갖고그것을닮음비라고한다이렇게공부했었는데.이것말고도이런것들도성립할거라예를들어작은삼각형에서빗변분의높이얼마입니까.큰삼각형에서빗변분의높이얼마죠숯불에유기적이들어때요.오분의삼으로서로같다라는확인할수있고또마찬가지로자는삼각형에서예를들어서빗변분의밑변에봅시다오분의사적큰삼각형에서변분에밑변에보면얼마죠십분에반이죠근데.이것도사러때같아요.마찬가지로하나더해볼까요.작은삼각형에서밑변불에높이는사분의삼이적큰삼각형에서미평그래도필요없대요.팔분의육인데.이들이모두다어때같다란걸려놈들이알수있단얘기야결국지금이둘의차이는같은거같아보이나요.달라요.작은삼각형과큰삼각형의두대는변의길이비를이야기한게닮음비고일삼각형에서어떤두병을뽑아서그들의비람큰삼각형둘레길이팔미터서로비가서로같은성질이있다라는게바로닮은직각삼각형에서성질입니다.이제우리가요.내용은당연히받아들일수있을테니이내용에대해서간단하게확인문자한번살펴보고가도록하겠습니다.오른쪽그림과같은삼각형에이비씨안에있는작은단답형큰삼각형에이디가있습니다.답을구하세요라고했는데.저기큰삼각형에서지금길을알수가없어요.그죠작은삼각형만주어져있는데.구하라는건못가고갔더니에이디브이에이엔티에이쥐이이브레디얼굴큰삼각형에서빗변분의높이를구하라얘기네근데.그건우리가결국눈물을구하는것과같다자는삼각형에서소피아서울어때같다라는이야기입니다.왜두직각삼각형은서로닮은이니까.결국이거뭐라고쓸수있어요.바로가볼게요.빗변분의높이였으니견본의높이이렇게해주면루트이의제곱유리가에서이분의루트이이렇게사람들이구할수있다는겁니다.하나더두번째건뭘바라그랬으니까에이이마이너스팔잊어에이이분해분해에이티아하빗변분의밑변이래왜나와있지않지만우리는할수있다작은직각삼각형에서빗변분의밑변을해주면되니까루트이분의일이렇게계속구해서유리한가지만해주면바로답을구할수있다라는이야기해요.어렵나요.아직까지내용은전혀어렵지않죠자이제우리가삼각비뜻바로오늘핵심입니다.오늘핵심개념삼각비뜻에대해서여러분들께서실행을문제집이나아니면다른여러가지학습방법을통해서공부를했을텐데.뭐사인코사인탄젠트라는걸들어본친구들도있을거고요.

In [24]:
kkma = Kkma()

In [25]:
ebs_pos = kkma.pos(ko_pre_EBS)

In [26]:
google_pos = kkma.pos(ko_pre_GOOGLE)

In [27]:
def selected_pos(pos_text):
    alist = []
    for _ in pos_text:
        if _[1] in ['NNG','NNP','NNB','NR','NP', 'VV','VA','XR']:
            alist.append(_[0])
            continue
    return alist

# 'NNG' : 보통명사,
# 'NNP': 고유명사,
# 'NNB': 일반의존명사,
# 'NR': 수사,
# 'NP': 대명사, 
# 'VV': 동사,
# 'VA': 형용사,
# 'XR': 어근

In [28]:
list_ebs_pos = selected_pos(ebs_pos)

In [29]:
list_google_pos = selected_pos(google_pos)

In [30]:
list_ebs_pos

['반갑',
 '이비',
 '에스',
 '친구',
 '여러분',
 '여러분',
 '중학',
 '뉴런',
 '수학',
 '삼하',
 '강의',
 '되',
 '손',
 '석민',
 '이',
 '학년',
 '뉴런',
 '통하',
 '서도',
 '만나',
 '여러분',
 '이렇',
 '만나',
 '되',
 '반갑',
 '앞',
 '삼하',
 '과정',
 '어떻',
 '진행',
 '지',
 '대하',
 '여러분',
 '오리엔테이션',
 '강의',
 '통하',
 '확인',
 '수',
 '있',
 '거',
 '오리엔테이션',
 '강의',
 '듣',
 '오시',
 '여러분',
 '삼',
 '학년',
 '과정',
 '이하',
 '과정',
 '어떻',
 '수업',
 '확인',
 '수',
 '있',
 '것',
 '삼',
 '학년',
 '과정',
 '같',
 '경우',
 '크',
 '삼개',
 '대단원',
 '이루어지',
 '번째',
 '삼각비',
 '단원',
 '번째',
 '원',
 '성질',
 '번째',
 '통계',
 '이루어지',
 '우리',
 '일',
 '학년',
 '마',
 '찬가',
 '지',
 '학년',
 '그리하',
 '도형',
 '대하',
 '이야기',
 '삼하',
 '다루',
 '구',
 '나라',
 '걸',
 '생각',
 '되',
 '터',
 '오늘',
 '시간',
 '삼각비',
 '삼각',
 '알',
 '비',
 '알',
 '그',
 '합',
 '쳐지',
 '삼각비',
 '새',
 '용어',
 '나오',
 '그러',
 '개념',
 '공부',
 '앞',
 '서',
 '삼각비',
 '뜻',
 '시작',
 '것',
 '시간',
 '오늘',
 '시간',
 '시작',
 '해보',
 '하',
 '터',
 '중학교',
 '삼',
 '학년',
 '수학',
 '손',
 '샘',
 '끝내',
 '공부',
 '내용',
 '가지',
 '개념',
 '오늘',
 '가지',
 '개념',
 '번째',
 '제목',
 '긷',
 '닮',
 '직각',
 '삼각형',
 '대응변',
 '길이',


In [31]:
list_google_pos

['반갑',
 '이비',
 '에스',
 '여러분',
 '여러분',
 '중학',
 '뉴런',
 '수학',
 '삼',
 '강의',
 '되',
 '손',
 '성',
 '민',
 '이',
 '학년',
 '이렇',
 '을',
 '통해',
 '만나',
 '여러분',
 '이렇',
 '만나',
 '되',
 '반갑',
 '앞',
 '삼과',
 '정',
 '어떻',
 '진행',
 '지',
 '대하',
 '여러분',
 '오리엔테이션',
 '강의',
 '통하',
 '확인',
 '수',
 '있',
 '거구',
 '오리엔테이션',
 '강의',
 '듣',
 '오시',
 '여러분',
 '삼',
 '학년',
 '과정',
 '하마',
 '종이',
 '접',
 '어떻',
 '수업',
 '확인',
 '수',
 '있',
 '것',
 '삼',
 '학년',
 '가정',
 '같',
 '경우',
 '크',
 '대단원',
 '이루어지',
 '번째',
 '삼각비',
 '단어',
 '이거',
 '번째',
 '원',
 '성질',
 '번째',
 '통계',
 '이루어지',
 '우리',
 '일',
 '학년',
 '마',
 '찬가',
 '지',
 '학년',
 '그리하',
 '도형',
 '대하',
 '이야기',
 '삼화',
 '다루',
 '구',
 '나라',
 '걸',
 '생각',
 '되',
 '터',
 '오늘',
 '시간',
 '삼각비',
 '삼각',
 '알',
 '알',
 '그',
 '합',
 '쳐지',
 '삼각비',
 '새',
 '영화',
 '녀',
 '공부',
 '서',
 '삼각비',
 '뜻',
 '시작',
 '것',
 '시간',
 '오늘',
 '차',
 '시간',
 '시작',
 '해보',
 '하',
 '터',
 '중학교',
 '삼',
 '학년',
 '수학',
 '그',
 '끝내',
 '공부',
 '내용',
 '가지',
 '개념',
 '자',
 '오늘',
 '가지',
 '번째',
 '제목',
 '긷',
 '닮',
 '집값',
 '삼각형',
 '대응변',
 '길이',
 '피나',
 '것',
 '있',
 '번째',


In [32]:
integrated_into_W2V = [list_ebs_pos + list_google_pos]

In [33]:
model_skip = Word2Vec(
    integrated_into_W2V, 
    size = 100,
    sg = 1
)
words_skip = list(model_skip.wv.vocab)
print(words_skip)

['반갑', '이비', '에스', '친구', '여러분', '중학', '뉴런', '수학', '삼하', '강의', '되', '손', '이', '학년', '통하', '서도', '만나', '이렇', '앞', '과정', '어떻', '진행', '지', '대하', '오리엔테이션', '확인', '수', '있', '거', '듣', '오시', '삼', '이하', '수업', '것', '같', '경우', '크', '삼개', '이루어지', '번째', '삼각비', '단원', '원', '성질', '우리', '일', '마', '찬가', '그리하', '도형', '이야기', '다루', '구', '나라', '걸', '생각', '터', '오늘', '시간', '삼각', '알', '비', '그', '합', '쳐지', '새', '용어', '나오', '그러', '개념', '공부', '서', '뜻', '시작', '해보', '하', '중학교', '끝내', '내용', '가지', '제목', '긷', '닮', '직각', '삼각형', '길이', '그다음', '뭐', '면', '갖', '기본적', '가', '자', '리', '다라', '받아들이', '그렇', '고하', '그리', '두개', '편하', '제일', '좋', '방법', '무엇', '까', '한번', '여기', '수선', '긋', '바깥쪽', '보이', '다르', '빗변', '위', '점', '잡', '그어주', '직', '에이', '비씨', '에이디', '닮음', '때', '배우', '통', '둘', '중이', '닮음비', '변', '만약', '오라', '십', '대이', '가이', '사', '노', '얼마', '배', '팔', '육', '작', '쪽', '일대', '대일', '삼이', '육사', '이것', '다일', '대', '이로', '그것', '말', '이러하', '드', '얘기', '예', '분의', '높이', '어', '서로', '마찬가지', '밑변', '하나', '사분', '팔분의', '지금', '차이', '뽑', '비가', '간단', '문제

In [34]:
model_skip.wv.similar_by_word('콜사인',topn=20)

[('기서', 0.34499046206474304),
 ('밥', 0.32109513878822327),
 ('맵', 0.2858135402202606),
 ('베', 0.27728375792503357),
 ('엔', 0.2662128210067749),
 ('대부분', 0.25855526328086853),
 ('소리', 0.2512451112270355),
 ('에이큐', 0.24773365259170532),
 ('지면', 0.24631622433662415),
 ('잔소리', 0.24409186840057373),
 ('애인', 0.24266517162322998),
 ('꺼내', 0.230984166264534),
 ('이리', 0.22866399586200714),
 ('근', 0.22720588743686676),
 ('무언가', 0.22291520237922668),
 ('도임', 0.22079870104789734),
 ('기분', 0.21939745545387268),
 ('십삼도', 0.2177012711763382),
 ('파', 0.20619776844978333),
 ('별곡', 0.2038392871618271)]

In [35]:
model_skip.wv.similar_by_word('데이트',topn=20) #대입

[('관', 0.33266690373420715),
 ('최선', 0.30536869168281555),
 ('신나', 0.29843634366989136),
 ('변의', 0.28091737627983093),
 ('번가', 0.2695499658584595),
 ('별표', 0.26810845732688904),
 ('회의', 0.25756731629371643),
 ('잔소리', 0.2533544600009918),
 ('오케이', 0.2515057921409607),
 ('영일', 0.24736937880516052),
 ('세', 0.24210885167121887),
 ('필기', 0.23895637691020966),
 ('목표', 0.2387159913778305),
 ('아홉', 0.235133558511734),
 ('이모', 0.23428763449192047),
 ('지하', 0.23065698146820068),
 ('전화', 0.2280419021844864),
 ('접', 0.226191908121109),
 ('계속', 0.22542394697666168),
 ('하자', 0.2252076417207718)]

In [36]:
model_skip.wv.similar_by_word('직각',topn=20)

[('삼각형', 0.999163806438446),
 ('같', 0.9969443082809448),
 ('크', 0.9961546063423157),
 ('이야기', 0.9959684610366821),
 ('닮음', 0.9956910610198975),
 ('우리', 0.9947950839996338),
 ('그리', 0.9946128129959106),
 ('만들', 0.9946068525314331),
 ('긋', 0.9944998621940613),
 ('이등변', 0.9944286346435547),
 ('닮', 0.9940715432167053),
 ('오른쪽', 0.9940007925033569),
 ('가지', 0.9938973784446716),
 ('보이', 0.9938529133796692),
 ('말', 0.9938239455223083),
 ('작', 0.9938101768493652),
 ('경우', 0.9937812685966492),
 ('편하', 0.9937397241592407),
 ('서', 0.9937055110931396),
 ('만약', 0.9936410188674927)]

In [37]:
model_skip.wv.similar_by_word('양변',topn=20)

[('이루', 0.9996341466903687),
 ('자리', 0.9995631575584412),
 ('계산', 0.9995502829551697),
 ('쓰', 0.9995362758636475),
 ('곱하기', 0.9995229244232178),
 ('이렇', 0.9994860291481018),
 ('같아지', 0.9994803071022034),
 ('다이', 0.9994785189628601),
 ('가깝', 0.9994761347770691),
 ('뭐하', 0.9994736313819885),
 ('삼일', 0.9994714260101318),
 ('결과', 0.9994698762893677),
 ('분자', 0.9994691014289856),
 ('배', 0.9994670748710632),
 ('치', 0.9994667172431946),
 ('영이', 0.999466598033905),
 ('오이', 0.9994431138038635),
 ('간단', 0.9994357228279114),
 ('성립', 0.9994279742240906),
 ('중심', 0.999426543712616)]

In [38]:
model_skip.wv.similar_by_word('양배추',topn=20)

KeyError: "word '양배추' not in vocabulary"

In [39]:
model_skip.wv.similar_by_word('쓰레기',topn=20) #직'선의기'울기

[('사이즈', 0.2701641321182251),
 ('사부', 0.2608288824558258),
 ('양면', 0.24636995792388916),
 ('세상', 0.24512770771980286),
 ('편이', 0.2421935647726059),
 ('따라하', 0.23888084292411804),
 ('집', 0.2368256151676178),
 ('표시', 0.22822050750255585),
 ('육십삼', 0.2145659476518631),
 ('진짜', 0.21376700699329376),
 ('귀', 0.21149569749832153),
 ('요기', 0.21009370684623718),
 ('이천', 0.20899270474910736),
 ('채널', 0.20862352848052979),
 ('혼자', 0.2035643458366394),
 ('하루', 0.20328816771507263),
 ('햇빛', 0.1990417242050171),
 ('루피', 0.19708818197250366),
 ('개가', 0.19522418081760406),
 ('아무것', 0.1940845251083374)]

In [40]:
model_skip.wv.similar_by_word('싸이',topn=20)

[('도움', 0.284671813249588),
 ('번가', 0.27133652567863464),
 ('이별', 0.26752209663391113),
 ('니은', 0.26674050092697144),
 ('단', 0.2568521499633789),
 ('블랙', 0.2535267770290375),
 ('부러지', 0.25317224860191345),
 ('각둘', 0.2492346465587616),
 ('십삼도', 0.24677743017673492),
 ('별거', 0.2437758445739746),
 ('등', 0.24097999930381775),
 ('기계', 0.23372557759284973),
 ('판', 0.22546763718128204),
 ('실제', 0.22273766994476318),
 ('갑', 0.22187486290931702),
 ('막연', 0.21994319558143616),
 ('닿', 0.20701295137405396),
 ('못', 0.20184728503227234),
 ('아이거', 0.20130261778831482),
 ('칸', 0.19988811016082764)]

In [41]:
model_skip.wv.similar_by_word('임진각',topn=20) #끼인각

[('미터', 0.3011842668056488),
 ('관심사', 0.29812368750572205),
 ('부치', 0.24357907474040985),
 ('사다리꼴', 0.24316370487213135),
 ('에이스', 0.24243761599063873),
 ('가져오', 0.24143216013908386),
 ('위치', 0.2345166802406311),
 ('유용', 0.23409633338451385),
 ('책', 0.23120582103729248),
 ('너', 0.2293131798505783),
 ('편의점', 0.22594745457172394),
 ('일반', 0.2249520719051361),
 ('구사', 0.22359813749790192),
 ('나눠주', 0.2152978628873825),
 ('엘', 0.21365229785442352),
 ('이부', 0.20826616883277893),
 ('옷', 0.20437581837177277),
 ('에이지', 0.20182321965694427),
 ('써', 0.19824887812137604),
 ('회의', 0.19064773619174957)]

# 아래는 LSTM wikidocs

In [ ]:
EBSlist_tokens = [' '.join(Text(ma(_)).tokens) for _ in new_textE1]

In [ ]:
EBSlist_tokens

In [ ]:
t = Tokenizer()
t.fit_on_texts(morphed_EBS)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

In [ ]:
sequences = list()

for line in morphed_EBS: # 1,214 개의 샘플에 대해서 샘플을 1개씩 가져온다.
    encoded = t.texts_to_sequences([line])[0] # 각 샘플에 대한 정수 인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11] # 11개의 샘플 출력

In [ ]:
print(t.word_index)

In [ ]:
index_to_word={}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 582번 단어 : {}'.format(index_to_word[582]))

In [ ]:
max_len=max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

In [ ]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

In [ ]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [ ]:
print(X[:3])

In [ ]:
print(y[:3]) # 레이블

In [ ]:
vocab_size

In [ ]:
y = to_categorical(y, num_classes=vocab_size)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
# y데이터를 분리하였으므로 이제 X데이터의 길이는 기존 데이터의 길이 - 1
model.add(LSTM(100))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=10, verbose=2)

In [ ]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=23, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [ ]:
sentence_generation(model, t, '이 병 분에 높이', 5)